In [1]:
import gym
import matplotlib.pyplot as plt
import collections
import numpy as np

* 알고리즘

In [335]:
state_bag = []

class lake_tree():
    def __init__(
        self,
        name: str,
        v_val: float,
        transit: dict,
        action_hist: dict,
    ):
        self.name = name 
        self.v_val = v_val
        self.transit = transit
        self.action_hist = action_hist

    def get_action(self, state_bag_list,is_random: bool):
        if is_random:
            a = np.random.choice([0,1,2,3])
            return a
        else:
            best_action = None
            best_action_val = None
            for key in list(self.action_hist.keys()):
                action_value = 0
                total_n = 0
                for small_key in list(self.action_hist[key].keys()):
                    sub_n = self.action_hist[key][small_key]
                    node_v_val = get_state_tree(small_key, state_bag_list).v_val

                    total_n += sub_n
                    action_value += sub_n * node_v_val
                action_value /= total_n

                if best_action is None:
                    best_action = key
                    best_action_val = action_value
                
                elif best_action_val < action_value:
                    best_action = key
                    best_action_val = action_value

            return best_action
        
    def v_update(self):
        t_count = 0
        t_val_sum_ratio = 0
        for t_tree in list(self.transit.keys()):
            
            t_count += self.transit[t_tree]
            t_val_sum_ratio += t_tree.v_val * self.transit[t_tree]
        
        if self.v_val != 1:
            t_val_sum_ratio /= t_count
            self.v_val = t_val_sum_ratio




def get_state_tree(state: str, tree_list: list):
    state_tree = None
    for i in range(len(tree_list)):
        if tree_list[i].name == state:
            state_tree = tree_list[i]
            return  state_tree
    if state_tree is None:
        return 'notree'


eps = 1
eps_grad = 0.001
# np.random.random()

env = gym.make("FrozenLake-v1", is_slippery = True)
s, prob = env.reset()
init_v_val = 0

state_bag.append(lake_tree(name = str(s), v_val = init_v_val, transit=dict(),action_hist=dict()))

i_tree = get_state_tree(str(s), state_bag)
a = i_tree.get_action(state_bag,is_random = True)

for i in range(7000):
    eps *= (1-eps_grad)
    
    ns, r, done, info, whwh = env.step(a)
    try:
        i_tree.action_hist[a][str(ns)] += 1 
    except:
        try:
            i_tree.action_hist[a][str(ns)] = 1
        except:
            i_tree.action_hist[a] = dict()
            i_tree.action_hist[a][str(ns)] = 1

    new_tree = get_state_tree(str(ns), state_bag)
    if new_tree == 'notree':

        state_bag.append(lake_tree(name = str(ns), v_val = init_v_val, transit=dict(),action_hist=dict()))
        new_tree = get_state_tree(str(ns), state_bag)
        i_tree.transit[new_tree] = 1
        i_tree.v_update()
        i_tree = new_tree
    else:
        try:
            i_tree.transit[new_tree] += 1
            i_tree.v_update()
            i_tree = new_tree
            # 들릴때마다 없데이트
            
        except:
            i_tree.transit[new_tree] = 1
            i_tree.v_update()
            i_tree = new_tree
            


    # print(ns, r, done, info, whwh)
    if done:
        if r == 0:
            env.reset()
            i_tree = get_state_tree(str(0), state_bag)

            if np.random.random() < eps:
                a = i_tree.get_action(state_bag,is_random = True)
            else:
                a = i_tree.get_action(state_bag,is_random = False)
                if a is None:
                    a = i_tree.get_action(state_bag,is_random = True)

            
        else:
            i_tree.v_val = 1
            env.reset()
            i_tree = get_state_tree(str(0), state_bag)
            if np.random.random() < eps:
                a = i_tree.get_action(state_bag,is_random = True)
            else:
                a = i_tree.get_action(state_bag,is_random = False)
                if a is None:
                    a = i_tree.get_action(state_bag,is_random = True)

            
            
    else:
        if np.random.random() < eps:
            a = i_tree.get_action(state_bag,is_random = True)
        else:
            a = i_tree.get_action(state_bag,is_random = False)
            if a is None:
                    a = i_tree.get_action(state_bag,is_random = True)

In [353]:
#성능확인
episode_m = 10
result = []

env = gym.make("FrozenLake-v1", is_slippery = True, render_mode="rgb_array")
s, prob = env.reset()
# env.render()
i_tree = get_state_tree(str(s), state_bag)
a = i_tree.get_action(state_bag,is_random = False)

for i in range(episode_m):
    while True:
        ns, r, done, info, whwh = env.step(a)
        new_tree = get_state_tree(str(ns), state_bag)
        a = new_tree.get_action(state_bag,is_random = False)
        if done:
            result.append(r)
            s, prob = env.reset()
            i_tree = get_state_tree(str(s), state_bag)
            a = i_tree.get_action(state_bag,is_random = False)
            break

print('성능: ', np.sum(result))
    

성능:  9.0
